In [1]:
import numpy as np
import pandas as pd

all_data = pd.read_csv('voice.csv')
features = [feat for feat in all_data.columns if feat != 'label']
output = 'label'
all_data[output].replace(['male','female'], [1, 0], inplace = True)
all_samples = all_data[features].values
labels = all_data[output].values

In [8]:
from sklearn.decomposition import PCA as sklearnPCA

sklearn_pca = sklearnPCA(n_components = 10)
sklearn_transf = sklearn_pca.fit_transform(all_samples)
print(sklearn_transf.shape)


(3168, 10)


In [10]:
transformed = sklearn_transf.T
#print(transformed.shape)
assert transformed.shape == (10,3168), "The matrix is not 10x3168 dimensional."
#print(transformed)
transformed = transformed.T

In [11]:
def nonlin(x, deriv = False):
    if (deriv == True):
       return x * (1 - x)
    else:
       return 1/(1 + np.exp(-x))

In [12]:
#Split dataset into training dat and test data

#all_data = pd.read_csv('voice.csv')
features = [feat for feat in all_data.columns if feat != 'label']
output = 'label'
num_datapoints = len(transformed)

#Training data is first 90% of male and female
#Test data is las 10% of male and female

test_male = int(1584 * 0.1)
test_female = int((num_datapoints-1584) * 0.1)

#all_data[output].replace(['male','female'], [1, 0], inplace = True)

train_data_male = transformed[0:(1583-test_male)]
train_data_female = transformed[1584:-test_female]
test_data_male = transformed[(1583 - test_male):1583]
test_data_female = transformed[-test_female:]

train_data = np.concatenate((train_data_male, train_data_female), axis = 0)
test_data = np.concatenate((test_data_male, test_data_female),axis = 0)

train_labels_male = all_data[output].iloc[0:(1583 - test_male)].values
train_labels_female = all_data[output].iloc[1584:-test_female].values
test_labels_male = all_data[output].iloc[(1583 - test_male):1583].values 
test_labels_female = all_data[output].iloc[-test_female:].values

train_labels = np.concatenate((train_labels_male, train_labels_female), axis = 0)
test_labels = np.concatenate((test_labels_male, test_labels_female), axis = 0)

print(len(train_data))
print(len(test_data))

2851
316


In [21]:
#To ensure random number of every loop is the same
np.random.seed(10)
#Initialize weights
syn0 = 2 * np.random.random((10,1)) - 1
l0 = train_data
#Keep updating weights
for iter in range(100000):
    #l0 = train_data
    #Input(l0) with weights(syn0) 
    #Combined by sigmoid function
    l1 = nonlin(np.dot(l0,syn0))
    
    #How different that l1 from exact labels
    l1_error = train_labels.T - l1.T
    
    #l1_delta = l1_error.T * nonlin(l1, True)
    #syn0 += np.dot(l0.T, l1_delta)
    syn0 += np.dot(l0.T, l1_error.T)
print(l1)

[[ 1.]
 [ 1.]
 [ 1.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]


In [22]:
count_0 = 0
count_1 = 0
for i in l1:
    if 0 in i:
       count_0 += 1
    else:
       count_1 += 1
print("predicted number of female", count_0)
print("predicted number of male", count_1)
print("total number of train data:",count_0 + count_1)

predicted number of female 2658
predicted number of male 193
total number of train data: 2851


In [23]:
correct_predic = 0
wrong_predic = 0
for i in range(0,2851):
    if train_labels[i] == l1[i]:
       correct_predic += 1
    else:
       wrong_predic += 1
    
print("Number of correct prediction", correct_predic)
print("Number of wrong prediction", wrong_predic)

Number of correct prediction 1487
Number of wrong prediction 1364


In [24]:
print("Correct rate is: ")
print(correct_predic/(wrong_predic+correct_predic))

Correct rate is: 
0.5215713784636969


In [25]:
l0 = test_data
   
l1 = nonlin(np.dot(l0,syn0))

In [26]:
count_0 = 0
count_1 = 0
for i in l1:
    if 0 in i:
       count_0 += 1
    else:
       count_1 += 1
print("predicted number of female", count_0)
print("predicted number of male", count_1)
print("total number of train data:",count_0 + count_1)

predicted number of female 165
predicted number of male 151
total number of train data: 316


In [27]:
correct_predic = 0
wrong_predic = 0
for i in range(0,316):
    if test_labels[i] == l1[i]:
       correct_predic += 1
    else:
       wrong_predic += 1
    
print("Number of correct prediction", correct_predic)
print("Number of wrong prediction", wrong_predic)

Number of correct prediction 249
Number of wrong prediction 67


In [28]:
print("Correct rate is: ", correct_predic/(wrong_predic + correct_predic))

Correct rate is:  0.7879746835443038
